In [ ]:
!pip install reportlab
!pip install pillow

In [ ]:
# Calibration certificate 
# Written by: Diran Edwaarrds on 16/11/2024
# Last modified: by Diran Edwaarrds on 16/11/2024
# License GNU 3.0

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from datetime import datetime, timedelta
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch
import subprocess
import os

# Calibration metadata
certificate_number = "20241116"
issue_date = datetime.now()
recalibration_date = issue_date + timedelta(days=365)  # Assuming annual calibration
description = "Temperature Probe - Indicator"
serial_number = "heston blumenthal 0001"
manufacturer = "Salter"
calibrated_by = "Diran Edwards"
max_permissible_error = 0.5  # ±0.5°C

# Sample data (replace with your actual measurements)
reference_temps = np.array([3.1, 50.1, 76.2, 80.5])
measured_temps = np.array([3.9, 50.7, 76.7, 81.8])

# Calculate error
error = measured_temps - reference_temps

# Perform linear regression
slope, intercept, r_value, p_value, std_err = stats.linregress(reference_temps, measured_temps)

# Calculate linearity (R-squared value)
linearity = r_value**2

# Generate points for the best fit line
line = slope * reference_temps + intercept

# Uncertainty calculation
u_reference = 0.1  # Uncertainty of reference thermometer (example value)
u_resolution = 0.05  # Resolution uncertainty (example value)
u_repeatability = np.std(error)  # Standard deviation of errors
u_drift = 0.1  # Drift uncertainty (example value)

# Combined standard uncertainty
u_c = np.sqrt(u_reference**2 + u_resolution**2 + u_repeatability**2 + u_drift**2)

# Expanded uncertainty (k=2 for approximately 95% confidence level)
U = 2 * u_c

# Create the plot
plt.figure(figsize=(10, 6))
plt.scatter(reference_temps, measured_temps, color='blue', label='Measured vs Reference')
plt.plot(reference_temps, line, color='red', label=f'Best Fit Line (y = {slope:.4f}x + {intercept:.4f})')
plt.plot(reference_temps, reference_temps, color='green', linestyle='--', label='Ideal 1:1 Line')
plt.xlabel('Reference Temperature (°C)')
plt.ylabel('Measured Temperature (°C)')
plt.title('Thermometer Calibration')
plt.legend()
plt.grid(True)

# Save the plot
plt.savefig('thermometer_calibration_plot.png')
plt.close()

# Create PDF
pdf_filename = 'thermometer_calibration_certificate.pdf'
doc = SimpleDocTemplate(pdf_filename, pagesize=letter)
styles = getSampleStyleSheet()
elements = []

# Title
elements.append(Paragraph("CERTIFICATE of CALIBRATION", styles['Title']))
elements.append(Spacer(1, 0.25*inch))

# Metadata
metadata = [
    ["Issued By:", "Edwards Scientific Limited"],
    ["Date Of Issue:", issue_date.strftime('%d %B %Y')],
    ["Certificate Number:", certificate_number],
    ["Description:", description],
    ["Serial Number:", serial_number],
    ["Manufacturer:", manufacturer],
    ["Calibrated By:", calibrated_by],
    ["Calibration Date:", issue_date.strftime('%d %B %Y')],
    ["Re Calibration:", recalibration_date.strftime('%B %Y')],
]

metadata_table = Table(metadata)
metadata_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (0, -1), colors.lightgrey),
    ('TEXTCOLOR', (0, 0), (-1, -1), colors.black),
    ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
    ('FONTNAME', (0, 0), (-1, -1), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, -1), 10),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
    ('BACKGROUND', (1, 1), (-1, -1), colors.beige),
    ('GRID', (0, 0), (-1, -1), 1, colors.black)
]))
elements.append(metadata_table)
elements.append(Spacer(1, 0.25*inch))

# Method of Test
elements.append(Paragraph("Method Of Test:", styles['Heading2']))
elements.append(Paragraph("The temperature scale used is the International Temperature Scale ITS-90", styles['Normal']))
elements.append(Paragraph("The Temperature Probe & Indicator has been calibrated using a calibrated reference master temperature indicator & probe which is traceable to National Standards.", styles['Normal']))
elements.append(Spacer(1, 0.25*inch))

# Insert the method image here
method_image_path = 'method_image.png'  # Replace with your image file path
if os.path.exists(method_image_path):
    img = Image(method_image_path)
    img.drawHeight = 6*inch  # Adjust the height as needed
    img.drawWidth = 6*inch   # Adjust the width as needed
    elements.append(img)
    elements.append(Spacer(1, 0.25*inch))
else:
    print(f"Warning: Method image file '{method_image_path}' not found.")

elements.append(Paragraph(f"Measurement Uncertainty: ± {U:.2f}°C", styles['Normal']))
elements.append(Paragraph(f"Recommended Maximum Permissible Error: ± {max_permissible_error}°C", styles['Normal']))
elements.append(Spacer(1, 0.25*inch))

# Calibration Results
elements.append(Paragraph("Calibration Results:", styles['Heading2']))
data = [["Calibration Temperature (°C)", "Indicated Temperature (°C)", "Error (°C)"]]
for ref, meas, err in zip(reference_temps, measured_temps, error):
    data.append([f"{ref:.2f}", f"{meas:.1f}", f"{err:.1f}"])

results_table = Table(data)
results_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, -1), 10),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
    ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
    ('GRID', (0, 0), (-1, -1), 1, colors.black)
]))
elements.append(results_table)
elements.append(Spacer(1, 0.25*inch))

# Analysis
elements.append(Paragraph("Analysis:", styles['Heading2']))
analysis = [
    f"Linearity (R-squared): {linearity:.4f}",
    f"Slope: {slope:.4f}",
    f"Intercept (Offset): {intercept:.4f}°C",
    f"Mean Error: {np.mean(error):.4f}°C",
    f"Standard Deviation of Error: {np.std(error):.4f}°C",
    f"Maximum Error: {np.max(error):.4f}°C",
    f"Minimum Error: {np.min(error):.4f}°C",
]
for item in analysis:
    elements.append(Paragraph(item, styles['Normal']))

elements.append(Spacer(1, 0.25*inch))

# Uncertainty components
elements.append(Paragraph("Uncertainty Components:", styles['Heading2']))
uncertainty_components = [
    f"Reference thermometer uncertainty: ± {u_reference:.4f}°C",
    f"Resolution uncertainty: ± {u_resolution:.4f}°C",
    f"Repeatability uncertainty: ± {u_repeatability:.4f}°C",
    f"Drift uncertainty: ± {u_drift:.4f}°C",
    f"Combined standard uncertainty: ± {u_c:.4f}°C",
    f"Expanded uncertainty (k=2): ± {U:.4f}°C"
]
for item in uncertainty_components:
    elements.append(Paragraph(item, styles['Normal']))

elements.append(Spacer(1, 0.25*inch))

# Uncertainty statement
elements.append(Paragraph("The reported expanded uncertainty is based on a standard uncertainty multiplied by a coverage factor k = 2, providing a level of confidence of approximately 95%.", styles['Normal']))

# Add the plot
elements.append(Spacer(1, 0.25*inch))
elements.append(Image('thermometer_calibration_plot.png', width=6*inch, height=4*inch))

# Build the PDF
doc.build(elements)

print(f"Calibration complete. Certificate saved as '{pdf_filename}'")

# Open the PDF
pdf_path = os.path.abspath(pdf_filename)
try:
    subprocess.run(['evince', pdf_path], check=True)
except subprocess.CalledProcessError:
    print(f"Failed to open PDF. File created at: {pdf_path}")
    print("You can open it manually using the 'Files' app in your Chromebook.")
except FileNotFoundError:
    print("PDF viewer (evince) not found. Please install it or open the PDF manually.")
    print(f"PDF file created at: {pdf_path}")


Calibration complete. Certificate saved as 'thermometer_calibration_certificate.pdf'


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from datetime import datetime, timedelta
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch
import subprocess
import os
from PIL import Image as PILImage
from reportlab.lib.utils import ImageReader
import io

from PIL import Image as PILImage
from reportlab.lib.utils import ImageReader
from reportlab.platypus import Image as ReportLabImage
import io

def create_image(image_path, width=3*inch):
    if os.path.exists(image_path):
        pil_img = PILImage.open(image_path)
        if pil_img.mode != 'RGB':
            pil_img = pil_img.convert('RGB')
        img_buffer = io.BytesIO()
        pil_img.save(img_buffer, format='PNG')
        img_buffer.seek(0)
        img = ReportLabImage(img_buffer, width=width)
        img.drawHeight = width * pil_img.height / pil_img.width  # maintain aspect ratio
        return img
    else:
        print(f"Warning: Image file '{image_path}' not found.")
        return None

# Calibration metadata
certificate_number = "20241116"
issue_date = datetime.now()
recalibration_date = issue_date + timedelta(days=365)  # Assuming annual calibration
description = "Temperature Probe - Indicator"
serial_number = "heston blumenthal 0001"
manufacturer = "Salter"
calibrated_by = "Diran Edwards"
max_permissible_error = 0.5  # ±0.5°C

# Sample data (replace with your actual measurements)
reference_temps = np.array([3.1, 50.1, 76.2, 80.5])
measured_temps = np.array([3.9, 50.7, 76.7, 81.8])

# Calculate error
error = measured_temps - reference_temps

# Perform linear regression
slope, intercept, r_value, p_value, std_err = stats.linregress(reference_temps, measured_temps)

# Calculate linearity (R-squared value)
linearity = r_value**2

# Generate points for the best fit line
line = slope * reference_temps + intercept

# Uncertainty calculation
u_reference = 0.1  # Uncertainty of reference thermometer (example value)
u_resolution = 0.05  # Resolution uncertainty (example value)
u_repeatability = np.std(error)  # Standard deviation of errors
u_drift = 0.1  # Drift uncertainty (example value)

# Combined standard uncertainty
u_c = np.sqrt(u_reference**2 + u_resolution**2 + u_repeatability**2 + u_drift**2)

# Expanded uncertainty (k=2 for approximately 95% confidence level)
U = 2 * u_c

# Create the plot
plt.figure(figsize=(10, 6))
plt.scatter(reference_temps, measured_temps, color='blue', label='Measured vs Reference')
plt.plot(reference_temps, line, color='red', label=f'Best Fit Line (y = {slope:.4f}x + {intercept:.4f})')
plt.plot(reference_temps, reference_temps, color='green', linestyle='--', label='Ideal 1:1 Line')
plt.xlabel('Reference Temperature (°C)')
plt.ylabel('Measured Temperature (°C)')
plt.title('Thermometer Calibration')
plt.legend()
plt.grid(True)

# Save the plot
plt.savefig('thermometer_calibration_plot.png')
plt.close()

# Create PDF
pdf_filename = 'thermometer_calibration_certificate.pdf'
doc = SimpleDocTemplate(pdf_filename, pagesize=letter)
styles = getSampleStyleSheet()
elements = []

# Title
elements.append(Paragraph("CERTIFICATE of CALIBRATION", styles['Title']))
elements.append(Spacer(1, 0.25*inch))

# Metadata
metadata = [
    ["Issued By:", "Edwards Scientific Limited"],
    ["Date Of Issue:", issue_date.strftime('%d %B %Y')],
    ["Certificate Number:", certificate_number],
    ["Description:", description],
    ["Serial Number:", serial_number],
    ["Manufacturer:", manufacturer],
    ["Calibrated By:", calibrated_by],
    ["Calibration Date:", issue_date.strftime('%d %B %Y')],
    ["Re Calibration:", recalibration_date.strftime('%B %Y')],
]

metadata_table = Table(metadata)
metadata_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (0, -1), colors.lightgrey),
    ('TEXTCOLOR', (0, 0), (-1, -1), colors.black),
    ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
    ('FONTNAME', (0, 0), (-1, -1), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, -1), 10),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
    ('BACKGROUND', (1, 1), (-1, -1), colors.beige),
    ('GRID', (0, 0), (-1, -1), 1, colors.black)
]))
elements.append(metadata_table)
elements.append(Spacer(1, 0.25*inch))

# Method of Test
elements.append(Paragraph("Method Of Test:", styles['Heading2']))
elements.append(Paragraph("The temperature scale used is the International Temperature Scale ITS-90", styles['Normal']))
elements.append(Paragraph("The Temperature Probe & Indicator has been calibrated using a calibrated reference master temperature indicator & probe which is traceable to National Standards.", styles['Normal']))
elements.append(Spacer(1, 0.25*inch))

# Insert two images side by side
image_path1 = 'method_image1.png'  # Replace with your first image file path
image_path2 = 'method_image2.png'  # Replace with your second image file path

img1 = create_image(image_path1)
img2 = create_image(image_path2)

if img1 and img2:
    image_table = Table([[img1, img2]])
    image_table.setStyle(TableStyle([
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
    ]))
    elements.append(image_table)
    elements.append(Spacer(1, 0.25*inch))
else:
    print("Warning: One or both method images not found.")



elements.append(Paragraph(f"Measurement Uncertainty: ± {U:.2f}°C", styles['Normal']))
elements.append(Paragraph(f"Recommended Maximum Permissible Error: ± {max_permissible_error}°C", styles['Normal']))
elements.append(Spacer(1, 0.25*inch))

elements.append(Paragraph(f""))
elements.append(Paragraph(f""))
elements.append(Paragraph(f""))
elements.append(Paragraph(f""))
elements.append(Paragraph(f""))
elements.append(Paragraph(f""))
elements.append(Spacer(1, 0.25*inch))
elements.append(Paragraph(f""))
elements.append(Paragraph(f""))
elements.append(Spacer(1, 0.25*inch))
elements.append(Paragraph(f""))
elements.append(Paragraph(f""))
elements.append(Spacer(1, 0.25*inch))
# Calibration Results
elements.append(Paragraph("Calibration Results:", styles['Heading2']))
data = [["Calibration Temperature (°C)", "Indicated Temperature (°C)", "Error (°C)"]]
for ref, meas, err in zip(reference_temps, measured_temps, error):
    data.append([f"{ref:.2f}", f"{meas:.1f}", f"{err:.1f}"])

results_table = Table(data)
results_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, -1), 10),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
    ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
    ('GRID', (0, 0), (-1, -1), 1, colors.black)
]))
elements.append(results_table)
elements.append(Spacer(1, 0.25*inch))

# Analysis
elements.append(Paragraph("Analysis:", styles['Heading2']))
analysis = [
    f"Linearity (R-squared): {linearity:.4f}",
    f"Slope: {slope:.4f}",
    f"Intercept (Offset): {intercept:.4f}°C",
    f"Mean Error: {np.mean(error):.4f}°C",
    f"Standard Deviation of Error: {np.std(error):.4f}°C",
    f"Maximum Error: {np.max(error):.4f}°C",
    f"Minimum Error: {np.min(error):.4f}°C",
]
for item in analysis:
    elements.append(Paragraph(item, styles['Normal']))

elements.append(Spacer(1, 0.25*inch))

# Uncertainty components
elements.append(Paragraph("Uncertainty Components:", styles['Heading2']))
uncertainty_components = [
    f"Reference thermometer uncertainty: ± {u_reference:.4f}°C",
    f"Resolution uncertainty: ± {u_resolution:.4f}°C",
    f"Repeatability uncertainty: ± {u_repeatability:.4f}°C",
    f"Drift uncertainty: ± {u_drift:.4f}°C",
    f"Combined standard uncertainty: ± {u_c:.4f}°C",
    f"Expanded uncertainty (k=2): ± {U:.4f}°C"
]
for item in uncertainty_components:
    elements.append(Paragraph(item, styles['Normal']))

elements.append(Spacer(1, 0.25*inch))

# Uncertainty statement
elements.append(Paragraph("The reported expanded uncertainty is based on a standard uncertainty multiplied by a coverage factor k = 2, providing a level of confidence of approximately 95%.", styles['Normal']))

# Add the plot
elements.append(Spacer(1, 0.25*inch))
elements.append(Image('thermometer_calibration_plot.png', width=6*inch, height=4*inch))

# Build the PDF
doc.build(elements)

print(f"Calibration complete. Certificate saved as '{pdf_filename}'")

# Open the PDF
pdf_path = os.path.abspath(pdf_filename)
try:
    subprocess.run(['evince', pdf_path], check=True)
except subprocess.CalledProcessError:
    print(f"Failed to open PDF. File created at: {pdf_path}")
    print("You can open it manually using the 'Files' app in your Chromebook.")
except FileNotFoundError:
    print("PDF viewer (evince) not found. Please install it or open the PDF manually.")
    print(f"PDF file created at: {pdf_path}")

Calibration complete. Certificate saved as 'thermometer_calibration_certificate.pdf'
